In [ ]:
# Numpy for matrix operations, like MATLAB
import numpy as np

# naming the tensorflow import tf is standard practice
import tensorflow as tf

# We'll use the machine learning library sklearn to split our labeled data in two:
# one for training and one to verify our algorithm can predict labels of data it hasn't "seen"
from sklearn.model_selection import train_test_split

tf.logging.set_verbosity(tf.logging.INFO)

## Data preprocessing

In [ ]:
# pandas lets us conveniently load data from json into a table-like format called a DataFrame
import pandas as pd
path = "data/"
train = pd.read_json(path+"train.json")
test = pd.read_json(path+"test.json")

In [ ]:
# Extract the image data as numpy arrays, a format tensorflow can use
Xv1 = np.array([np.array(band).astype(np.float32) for band in train["band_1"]])
Xv2 = np.array([np.array(band).astype(np.float32) for band in train["band_2"]])
print("Xv1/2 shape: " + str(Xv1.shape))

# Put the predictions into a numpy array
vals = train.is_iceberg.values
Yv = np.asarray(vals, dtype=np.int32)

# Concatenate the two image channels into a single numpy array of three dimensions (a tensor!)
Xv_double = np.stack([Xv1,Xv2], axis = 2)
print("Xv_double shape: " + str(Xv_double.shape))

# Split the training data and labels in two separate parts
X_train, X_valid, y_train, y_valid = train_test_split(Xv_double, Yv, random_state=1, train_size=0.75, test_size=0.25)

print("Shape of training data: " + str(X_train.shape))
print("Shape of training labels : " + str(y_train.shape))
print("Shape of validation data : " + str(X_valid.shape))
print("Shape of validation labels : " + str(y_valid.shape))

## Defining the tensorflow input and model functions

In [ ]:
# Training data input function
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

In [ ]:
# Model function, where the magic happens
def model_fn(features, labels, mode):
    """
    Model function for CNN.
    Features and labels correspond to x and y from the train_input_fn we created above
    Mode decides whether to train the model, evaluate the accuracy of the model on labeled data, 
    or predict the category of unlabeled images
    
    """
    
    # Input Layer:
    # Store the image array as a tensorflow tensor.
    # tf.reshape will take our input, which is of dimensions [batch_size, vector_length = 75*75, num_channels = 2]
    input_layer = tf.reshape(features["x"], [-1, 75, 75, 2])
 
    # Add a convolutional layer of 32 [3,3] convolutions
    conv1 = tf.layers.conv2d(
    inputs = input_layer,
    filters=32,
    kernel_size=[3, 3],
    padding="same",
    activation=tf.nn.relu)
    
    # to use a dense (fully connected) layer, we need to flatten our inputs to vectors
    # We'll need to change the dimensions of flat_input if we add new layers above this line
    flat_input = tf.reshape(conv1, [-1, 75*75*32])

    # Dense Layer #1
    dense1 = tf.layers.dense(
        inputs=flat_input,
        units = 100,
        activation=tf.nn.relu)
    # Logits Layer
    
    # The logits layer, aka the output layer, outputs two values (one for ships, one for icebergs)
    logits = tf.layers.dense(inputs=dense1, units=2)

    
    predictions = {
        # Generate predictions (to be used in PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
    "accuracy": tf.metrics.accuracy(
    labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
    mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## Training the classifier on input data

In [ ]:
# Create the estimator object.
# The model function we created above, the model_dir lets tensorflow know where to store our neural network to disk.
# Note that when changing the architecture we'll need to change the model_dir
statoil_classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir="/tmp/00003")

In [ ]:
# Train the classifier using the input function
# Steps denotes the number of batches to train on. 
# The total number of images 'looked' at will be steps * batch_size (which is given in the input function)
statoil_classifier.train(
    input_fn=train_input_fn,
    steps=160)

## Evaluating our model locally

In [ ]:
# Evaluate the model and print results
valid_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": X_valid},
  y=y_valid,
  num_epochs=1,
  shuffle=False)
valid_results = statoil_classifier.evaluate(input_fn=valid_input_fn)
print(valid_results)

## Predicting on the unknown data and creating submission file

In [ ]:
Xt1 = np.array([np.array(band).astype(np.float32) for band in test["band_1"]])
Xt2 = np.array([np.array(band).astype(np.float32) for band in test["band_2"]])

# Concatenate the two image channels into a single numpy array
Xd_test = np.stack([Xt1,Xt2], axis = 1)
print("Xd_test: " + str(Xd_test.shape))

In [ ]:
# Create the input function with the test data
test_input_fn = tf.estimator.inputs.numpy_input_fn(
x={"x": Xd_test},
shuffle=False)

# Use the classifier to predict if the image is an iceberg or a ship
pred = statoil_classifier.predict(input_fn=test_input_fn)

In [ ]:
# predict returns a 'generator', store it as a list for convenience, and have a look at the data
pred_list = list(pred)
pred_list[:10]

In [ ]:
# Get the IDs needed for the submissions
id = test['id'].tolist()
# Get the 'probability' assigned that the image is an iceberg
is_iceberg = [i['probabilities'][1] for i in pred_list]

In [ ]:
# with zip we can add together two lists of the same length
ziplist = list(zip(id,is_iceberg))
res = []
# Concatenate the two lists to a single list of the form 'id,probability_iceberg'
for pair in ziplist:
    res.append(pair[0]+','+str(pair[1]))

In [ ]:
# write the result to a .csv file in the same folder as this notebook
with open('submission.csv', 'w') as f:
    f.write("id,is_iceberg\n")
    for item in res:
      f.write("%s\n" % item)